In [2]:
# Run in python console
import nltk; nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\malavika.suresh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy


In [21]:

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
data=pd.read_csv('D:\\PGP-BABI\\Capstone\\Airbnb\\Tasmania-detailed listings O.csv', error_bad_lines=False)

In [7]:
data_text = data[['house_rules']]

In [8]:
data_text['index'] = data_text.index

C:\Users\malavika.suresh\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
data_text.head

<bound method NDFrame.head of                                             house_rules  index
0     Just to reiterate it's a vegetarian household,...      0
1     No smoking inside the house. (A $20 linen char...      1
2     The house must be left in a clean and tidy con...      2
3     We appreciate guests who leave the house as ti...      3
4                                                   NaN      4
5     We ask our guests to smoke outside should they...      5
6     Just respect :) If you are looking for a 'part...      6
7     No smoking No parties or events No pets  Check...      7
8     Just respect :) If you are looking for a 'part...      8
9     Latest check in is by 8 p.m.  If you are a smo...      9
10    No Smoking, no drinking to excess, quiet after...     10
11    We're pretty easy going. I ask the following o...     11
12                                                  NaN     12
13    I expect my guests to look after my place as w...     13
14    No smoking inside o

In [10]:
documents = data_text

In [11]:
len(documents)

5073

In [13]:
# Convert to list
data = documents.house_rules.values.tolist()


In [14]:
data[0:5]

["Just to reiterate it's a vegetarian household, so no meat or fish at home please! I try to live an eco-friendly lifestyle, so I recycle, turn lights out when they're not needed, and compost fruit and veggie scraps and coffee grounds. I'd appreciate it if you could do the same. There is a compost tub in the cupboard under the sink. Please leave the communal areas tidy. If you cook, please clean up after yourself. There is a dishwasher, so either pop dirty dishes in there, or leave them on the kitchen bench for me to put in the dishwasher. The shampoo, conditioner and shower gel for guest use is at the end of the bath. Please don't use my other toiletries. If there is something else you need then just ask.,Likewise, please don't use bath towels other than those I have left for you. If you need more towels just ask. :-) Please make sure the back doors to the deck are locked when you leave (and it may seem like an obvious thing to say, but please make sure you shut the front door behind 

In [15]:
for sent in data:
    if type(sent)!=str:
        print(sent)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [16]:
data = [x for x in data if str(x) != 'nan']


In [17]:
    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

In [18]:
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [19]:
pprint(data[:1])

['Just to reiterate its a vegetarian household, so no meat or fish at home '
 'please! I try to live an eco-friendly lifestyle, so I recycle, turn lights '
 'out when theyre not needed, and compost fruit and veggie scraps and coffee '
 'grounds. Id appreciate it if you could do the same. There is a compost tub '
 'in the cupboard under the sink. Please leave the communal areas tidy. If you '
 'cook, please clean up after yourself. There is a dishwasher, so either pop '
 'dirty dishes in there, or leave them on the kitchen bench for me to put in '
 'the dishwasher. The shampoo, conditioner and shower gel for guest use is at '
 'the end of the bath. Please dont use my other toiletries. If there is '
 'something else you need then just ask.,Likewise, please dont use bath towels '
 'other than those I have left for you. If you need more towels just ask. :-) '
 'Please make sure the back doors to the deck are locked when you leave (and '
 'it may seem like an obvious thing to say, but pleas

In [20]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [21]:
data_words = list(sent_to_words(data))

print(data_words[:1])

[['just', 'to', 'reiterate', 'its', 'vegetarian', 'household', 'so', 'no', 'meat', 'or', 'fish', 'at', 'home', 'please', 'try', 'to', 'live', 'an', 'eco', 'friendly', 'lifestyle', 'so', 'recycle', 'turn', 'lights', 'out', 'when', 'theyre', 'not', 'needed', 'and', 'compost', 'fruit', 'and', 'veggie', 'scraps', 'and', 'coffee', 'grounds', 'id', 'appreciate', 'it', 'if', 'you', 'could', 'do', 'the', 'same', 'there', 'is', 'compost', 'tub', 'in', 'the', 'cupboard', 'under', 'the', 'sink', 'please', 'leave', 'the', 'communal', 'areas', 'tidy', 'if', 'you', 'cook', 'please', 'clean', 'up', 'after', 'yourself', 'there', 'is', 'dishwasher', 'so', 'either', 'pop', 'dirty', 'dishes', 'in', 'there', 'or', 'leave', 'them', 'on', 'the', 'kitchen', 'bench', 'for', 'me', 'to', 'put', 'in', 'the', 'dishwasher', 'the', 'shampoo', 'conditioner', 'and', 'shower', 'gel', 'for', 'guest', 'use', 'is', 'at', 'the', 'end', 'of', 'the', 'bath', 'please', 'dont', 'use', 'my', 'other', 'toiletries', 'if', 'there

In [22]:
print(data_words[:2])

[['just', 'to', 'reiterate', 'its', 'vegetarian', 'household', 'so', 'no', 'meat', 'or', 'fish', 'at', 'home', 'please', 'try', 'to', 'live', 'an', 'eco', 'friendly', 'lifestyle', 'so', 'recycle', 'turn', 'lights', 'out', 'when', 'theyre', 'not', 'needed', 'and', 'compost', 'fruit', 'and', 'veggie', 'scraps', 'and', 'coffee', 'grounds', 'id', 'appreciate', 'it', 'if', 'you', 'could', 'do', 'the', 'same', 'there', 'is', 'compost', 'tub', 'in', 'the', 'cupboard', 'under', 'the', 'sink', 'please', 'leave', 'the', 'communal', 'areas', 'tidy', 'if', 'you', 'cook', 'please', 'clean', 'up', 'after', 'yourself', 'there', 'is', 'dishwasher', 'so', 'either', 'pop', 'dirty', 'dishes', 'in', 'there', 'or', 'leave', 'them', 'on', 'the', 'kitchen', 'bench', 'for', 'me', 'to', 'put', 'in', 'the', 'dishwasher', 'the', 'shampoo', 'conditioner', 'and', 'shower', 'gel', 'for', 'guest', 'use', 'is', 'at', 'the', 'end', 'of', 'the', 'bath', 'please', 'dont', 'use', 'my', 'other', 'toiletries', 'if', 'there

In [23]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['just', 'to', 'reiterate', 'its', 'vegetarian', 'household', 'so', 'no', 'meat', 'or', 'fish', 'at', 'home', 'please', 'try', 'to', 'live', 'an', 'eco', 'friendly', 'lifestyle', 'so', 'recycle', 'turn', 'lights', 'out', 'when', 'theyre', 'not', 'needed', 'and', 'compost', 'fruit', 'and', 'veggie', 'scraps', 'and', 'coffee', 'grounds', 'id', 'appreciate', 'it', 'if', 'you', 'could', 'do', 'the', 'same', 'there', 'is', 'compost', 'tub', 'in', 'the', 'cupboard', 'under', 'the', 'sink', 'please', 'leave', 'the', 'communal_areas', 'tidy', 'if', 'you', 'cook', 'please', 'clean', 'up', 'after', 'yourself', 'there', 'is', 'dishwasher', 'so', 'either', 'pop', 'dirty', 'dishes', 'in', 'there', 'or', 'leave', 'them', 'on', 'the', 'kitchen', 'bench', 'for', 'me', 'to', 'put', 'in', 'the', 'dishwasher', 'the', 'shampoo', 'conditioner', 'and', 'shower', 'gel', 'for', 'guest', 'use', 'is', 'at', 'the', 'end', 'of', 'the', 'bath', 'please', 'dont', 'use', 'my', 'other', 'toiletries', 'if', 'there', '

In [24]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [25]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


In [26]:

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])



In [27]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['reiterate', 'vegetarian', 'household', 'meat', 'fish', 'home', 'try', 'live', 'eco', 'friendly', 'lifestyle', 'recycle', 'turn', 'light', 'be', 'need', 'compost', 'fruit', 'veggie', 'scrap', 'coffee', 'ground', 'would', 'appreciate', 'could', 'compost', 'tub', 'cupboard', 'sink', 'leave', 'communal_area', 'tidy', 'cook', 'clean', 'dishwasher', 'pop', 'dirty', 'dish', 'leave', 'kitchen', 'bench', 'put', 'dishwasher', 'shampoo', 'conditioner', 'shower', 'gel', 'guest', 'use', 'end', 'bath', 'do', 'not', 'use', 'toiletry', 'something', 'else', 'need', 'ask', 'likewise', 'do', 'not', 'use', 'bath', 'towel', 'leave', 'need', 'towel', 'ask', 'make_sure', 'back', 'door', 'deck', 'lock', 'leave', 'may', 'seem', 'obvious', 'thing', 'say', 'make_sure', 'shut', 'front_door']]


In [28]:
data_words_bigrams[:5]

[['reiterate',
  'vegetarian',
  'household',
  'meat',
  'fish',
  'home',
  'please',
  'try',
  'live',
  'eco',
  'friendly',
  'lifestyle',
  'recycle',
  'turn',
  'lights',
  'theyre',
  'needed',
  'compost',
  'fruit',
  'veggie',
  'scraps',
  'coffee',
  'grounds',
  'id',
  'appreciate',
  'could',
  'compost',
  'tub',
  'cupboard',
  'sink',
  'please',
  'leave',
  'communal_areas',
  'tidy',
  'cook',
  'please',
  'clean',
  'dishwasher',
  'either',
  'pop',
  'dirty',
  'dishes',
  'leave',
  'kitchen',
  'bench',
  'put',
  'dishwasher',
  'shampoo',
  'conditioner',
  'shower',
  'gel',
  'guest',
  'use',
  'end',
  'bath',
  'please',
  'dont',
  'use',
  'toiletries',
  'something',
  'else',
  'need',
  'ask',
  'likewise',
  'please',
  'dont',
  'use',
  'bath',
  'towels',
  'left',
  'need',
  'towels',
  'ask',
  'please',
  'make_sure',
  'back',
  'doors',
  'deck',
  'locked',
  'leave',
  'may',
  'seem',
  'like',
  'obvious',
  'thing',
  'say',
  'p

In [29]:
print(data_lemmatized[:5])

[['reiterate', 'vegetarian', 'household', 'meat', 'fish', 'home', 'try', 'live', 'eco', 'friendly', 'lifestyle', 'recycle', 'turn', 'light', 'be', 'need', 'compost', 'fruit', 'veggie', 'scrap', 'coffee', 'ground', 'would', 'appreciate', 'could', 'compost', 'tub', 'cupboard', 'sink', 'leave', 'communal_area', 'tidy', 'cook', 'clean', 'dishwasher', 'pop', 'dirty', 'dish', 'leave', 'kitchen', 'bench', 'put', 'dishwasher', 'shampoo', 'conditioner', 'shower', 'gel', 'guest', 'use', 'end', 'bath', 'do', 'not', 'use', 'toiletry', 'something', 'else', 'need', 'ask', 'likewise', 'do', 'not', 'use', 'bath', 'towel', 'leave', 'need', 'towel', 'ask', 'make_sure', 'back', 'door', 'deck', 'lock', 'leave', 'may', 'seem', 'obvious', 'thing', 'say', 'make_sure', 'shut', 'front_door'], ['smoke', 'house', 'linen', 'charge', 'use', 'second_bedroom', 'couple', 'staying'], ['house', 'must', 'leave', 'clean', 'tidy', 'condition', 'otherwise', 'extra', 'cleaning', 'fee', 'hour', 'apply', 'sorry', 'accept', 'p

In [ ]:
#TODO, use en_core_web_lg to take benefit of Word vectors.

In [22]:
#Load external file for lemmatized data using en_core_web_lg
import pickle
filename = 'D:\\PGP-BABI\\Capstone\\Airbnb\\LDA\\data_lemmatized.tar\\data_lemmatized.sav'
data_lemmatized = pickle.load(open(filename, 'rb'))

In [23]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

In [24]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [8]:
id2word[0]

'hotel'

In [33]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:3]]

[[('appreciate', 1),
  ('ask', 2),
  ('back', 1),
  ('bath', 2),
  ('be', 1),
  ('bench', 1),
  ('clean', 1),
  ('coffee', 1),
  ('communal_area', 1),
  ('compost', 2),
  ('conditioner', 1),
  ('cook', 1),
  ('could', 1),
  ('cupboard', 1),
  ('deck', 1),
  ('dirty', 1),
  ('dish', 1),
  ('dishwasher', 2),
  ('do', 2),
  ('door', 1),
  ('eco', 1),
  ('else', 1),
  ('end', 1),
  ('fish', 1),
  ('friendly', 1),
  ('front_door', 1),
  ('fruit', 1),
  ('gel', 1),
  ('ground', 1),
  ('guest', 1),
  ('home', 1),
  ('household', 1),
  ('kitchen', 1),
  ('leave', 4),
  ('lifestyle', 1),
  ('light', 1),
  ('likewise', 1),
  ('live', 1),
  ('lock', 1),
  ('make_sure', 2),
  ('may', 1),
  ('meat', 1),
  ('need', 3),
  ('not', 2),
  ('obvious', 1),
  ('pop', 1),
  ('put', 1),
  ('recycle', 1),
  ('reiterate', 1),
  ('say', 1),
  ('scrap', 1),
  ('seem', 1),
  ('shampoo', 1),
  ('shower', 1),
  ('shut', 1),
  ('sink', 1),
  ('something', 1),
  ('thing', 1),
  ('tidy', 1),
  ('toiletry', 1),
  ('tow

In [34]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [35]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.084*"house" + 0.056*"smoke" + 0.055*"pet" + 0.052*"shoe" + 0.047*"party" '
  '+ 0.040*"smoking" + 0.037*"respect" + 0.035*"allow" + 0.033*"child" + '
  '0.029*"area"'),
 (1,
  '0.104*"let" + 0.069*"know" + 0.061*"instruction" + 0.059*"work" + '
  '0.056*"immediately" + 0.042*"cutlery" + 0.040*"tv" + 0.033*"anything" + '
  '0.030*"possible" + 0.028*"replace"'),
 (2,
  '0.123*"do" + 0.104*"not" + 0.049*"front_door" + 0.041*"floor" + '
  '0.035*"rain" + 0.034*"recycle" + 0.031*"can" + 0.031*"report" + '
  '0.029*"help" + 0.029*"considerate"'),
 (3,
  '0.194*"guest" + 0.100*"property" + 0.060*"stay" + 0.054*"must" + '
  '0.045*"may" + 0.042*"charge" + 0.033*"fee" + 0.029*"book" + 0.028*"damage" '
  '+ 0.028*"additional"'),
 (4,
  '0.288*"home" + 0.140*"cottage" + 0.108*"treat" + 0.056*"neat" + '
  '0.048*"retreat" + 0.039*"special" + 0.038*"manner" + 0.022*"little" + '
  '0.013*"wildlife" + 0.011*"attend"'),
 (5,
  '0.128*"expect" + 0.083*"love" + 0.080*"responsibility" + 0.062*"

In [25]:
#Load external file for LDA models
import pickle
filename = 'D:\\PGP-BABI\\Capstone\\Airbnb\\LDA\\2_lda_models\\2_model_list_5_100.sav'
lda_models = pickle.load(open(filename, 'rb'))

In [7]:
lda_models[1]

In [36]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -10.465194472463816

Coherence Score:  0.44580986530296246


In [26]:
#Load external file for coherence of LDA models
import pickle
filename = 'D:\\PGP-BABI\\Capstone\\Airbnb\\LDA\\2_lda_models\\2_coh_score_5_100.sav'
coh_scores = pickle.load(open(filename, 'rb'))

In [27]:
# Compute Perplexity - looping through multiple models
#len(coh_scores)
coh_scores

[0.47630016469838327,
 0.4658526523837494,
 0.4619577045728525,
 0.477799299957476,
 0.4910453468534643,
 0.49697209698339656,
 0.4963152591809112,
 0.49379880641433915,
 0.48875307466587187,
 0.4897245087040179,
 0.49315310206927043,
 0.483200649269291,
 0.4851383636583307,
 0.4867465788264493,
 0.48762958282929464,
 0.48529873345129504,
 0.4803742763496622,
 0.4790301271747688,
 0.485227161953111,
 0.47876207730573306,
 0.4867119381324513,
 0.48058046383649244,
 0.47889062101801055,
 0.47865722847077674,
 0.4768517039012497,
 0.47865526746523435,
 0.4741099645833565,
 0.4759897338397737,
 0.4603694253027616,
 0.47539568216646677,
 0.4698525799007118,
 0.46674707514363833]

In [1]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

NameError: name 'pyLDAvis' is not defined

In [28]:
# Visualize the topics - looping through multiple models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_models[1], corpus, id2word)
vis

C:\Users\malavika.suresh\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.061604  0.056389       1        1  16.450535
1      0.051126  0.040140       2        1  15.968239
6     -0.046254 -0.025509       3        1  13.927762
3      0.049672 -0.041148       4        1  13.211642
5      0.058242 -0.041272       5        1  12.627998
4     -0.108896  0.059376       6        1  11.188785
0     -0.065728 -0.090276       7        1   8.751051
2      0.123442  0.042299       8        1   7.873984, topic_info=     Category           Freq               Term          Total  loglift  \
term                                                                      
134   Default  244253.000000              great  244253.000000  30.0000   
65    Default  131664.000000           location  131664.000000  29.0000   
145   Default  226394.000000              place  226394.000000  28.0000   
1     Default  241670.000000               stay  241670.000000  27.0000   
141   Default   72067.000000          recommend   72067.000000  26.0000   
47    Default   66878.000000               home   66878.000000  25.0000   
144   Default  122623.000000              clean  122623.000000  24.0000   
115   Default   72138.000000               walk   72138.000000  23.0000   
59    Default   28178.000000            kitchen   28178.000000  22.0000   
230   Default   99243.000000               host   99243.000000  21.0000   
156   Default   46533.000000                not   46533.000000  20.0000   
217   Default   32646.000000               view   32646.000000  19.0000   
142   Default   44719.000000         restaurant   44719.000000  18.0000   
224   Default   69374.000000               easy   69374.000000  17.0000   
135   Default   32533.000000             highly   32533.000000  16.0000   
29    Default   55953.000000           downtown   55953.000000  15.0000   
12    Default   23024.000000                bus   23024.000000  14.0000   
244   Default   66008.000000              space   66008.000000  13.0000   
422   Default   37350.000000              thank   37350.000000  12.0000   
342   Default   25692.000000             airbnb   25692.000000  11.0000   
149   Default   54785.000000              close   54785.000000  10.0000   
41    Default   57829.000000               good   57829.000000   9.0000   
122   Default   94681.000000              would   94681.000000   8.0000   
98    Default  120089.000000            seattle  120089.000000   7.0000   
132   Default   70375.000000         everything   70375.000000   6.0000   
343   Default   25147.000000                 do   25147.000000   5.0000   
355   Default   46171.000000              check   46171.000000   4.0000   
994   Default    9068.000000              value    9068.000000   3.0000   
100   Default   20608.000000               shop   20608.000000   2.0000   
130   Default   27970.000000           distance   27970.000000   1.0000   
...       ...            ...                ...            ...      ...   
224    Topic8   12341.993164               easy   69374.562500   0.8151   
1563   Topic8     872.364319             secure    2254.427490   1.5922   
1854   Topic8    2035.106812           building    6980.069336   1.3091   
466    Topic8    1535.527954          advertise    4887.624023   1.3838   
132    Topic8   10859.339844         everything   70375.156250   0.6728   
230    Topic8   13955.563477               host   99243.070312   0.5799   
140    Topic8    9044.553711            perfect   60229.441406   0.6456   
2761   Topic8     662.306763  convention_center    1664.830078   1.6199   
144    Topic8   13581.623047              clean  122623.695312   0.3412   
2747   Topic8    1603.313843               deck    5952.142090   1.2299   
125    Topic8    5134.869141            amazing   31961.583984   0.7131   
1755   Topic8    1786.801270         attraction    6982.558594   1.1786   
98     Topic8   12693.616211            se